In [ ]:
import pandas as pd  

data=pd.read_csv(r'C:\Users\Prthamesh\Downloads\archive (3)\list_attr_celeba.csv')

data

In [ ]:
col_list=data.columns.tolist()
col_list.remove('image_id')

In [ ]:
import pickle
with open('train_data.pkl','rb') as f: 
    train_files,train_labels = pickle.load(f)

In [ ]:
len(train_files[0])

In [ ]:
train_files[0]

In [ ]:
#intialising model for predicting the probabilities of each facial atrribute

from keras.models import Sequential
from keras.layers import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense


model = Sequential()
model.add(Conv2D(filters=32, kernel_size=2, padding='same', activation='relu', 
                        input_shape=(128, 128, 3)))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=128, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))

model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(500, activation='relu'))
model.add(Dense(2, activation='softmax'))
model.summary()

model.compile(loss='binary_crossentropy',
              optimizer='adam',metrics=['accuracy'])

In [ ]:
len(train_files)

In [ ]:
#from scipy import misc
import numpy as np
import cv2
from tqdm import tqdm
from keras.utils import np_utils
import imageio

path_d = r'C:\Users\Akash\Downloads\archive (3)'

def prepare_train(file_paths, labels):
    images = [imageio.imread(path_d + '\\' + path) for path in file_paths]
    new_img=[]
    for i in tqdm(range(len(images))):
        img=cv2.resize(images[i],(128,128))
        new_img.append(img)
    
     
    images=np.asarray(new_img)
    images=images/255

    labels=np.asarray(labels)
    labels=np_utils.to_categorical(labels,2)

    return images,labels

In [ ]:
# training the model (each model corresponds to 1 facial attribute)
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from tqdm import tqdm
for i in tqdm(range(40)):
    print(i, end = '\r')
    X_train,y_train=prepare_train(train_files[i],train_labels[i])
    print("Attribute: ",i)
    checkpointer = ModelCheckpoint(filepath='model.weights.best.hdf5', verbose=1, 
                               save_best_only=True)
    stop=EarlyStopping(monitor='val_loss',
                              min_delta=0,
                              patience=10,
                              verbose=1, mode='auto')
    hist = model.fit(X_train, y_train, batch_size=200, epochs=50,
          validation_split=0.2, callbacks=[checkpointer,stop], 
          verbose=2, shuffle=True) 
    model.load_weights('model.weights.best.hdf5')
    model.save('model'+str(i)+'.h5')

In [ ]:
import pandas as pd  

data=pd.read_csv(r'C:\Users\Akash\Downloads\archive (3)\list_attr_celeba.csv')

data.info()

In [ ]:
col_list=data.columns.tolist()
col_list.remove('image_id')

In [ ]:
import glob
import numpy as np
import imageio
import os.path as path
from scipy import misc

IMAGE_PATH = path_d + '\\' + 'img_align_celeba/img_align_celeba'
file_paths = glob.glob(path.join(IMAGE_PATH, '00*.jpg'))

images = [imageio.imread(path) for path in file_paths]
images = images[:2500]

In [ ]:
#import imageio
len(images)
#images = [imageio.imread('000026.jpg')]

In [ ]:
import cv2
import numpy as np
new_img=[]
for i in range(len(images)):
    img=cv2.resize(images[i],(128,128))
    new_img.append(img)
    
images=np.asarray(new_img)
images=images/255

In [ ]:
pred_arr = np.zeros((2500,2), dtype = float)

In [ ]:
from keras.utils import np_utils
from keras.models import load_model
import os.path as path
#
s=0
t=0
acc=[]
arr_index = 0

for j in [9]:
    print("\n\nAttribute: ",j)
    print(col_list[j])
    #labels=[]
    filename = path.basename(file_paths[i])
    index=int(filename.split(".")[0])
    labels=[]
    print(filename,index,end='\r')
    if data[col_list[j]][index-1]==1:
        labels.append(1)
    else:
        labels.append(0)
    labels=np.asarray(labels)
    labels=np_utils.to_categorical(labels,2)
    model = load_model('model'+str(j)+'.h5')
    #print(labels[0])
    preds=model.predict(images)
    #print(preds)
    #preds=np.round(preds, )
    #preds[preds>=0.5] = 1
    #preds[preds<0.5] = 0
    #score=np.sum(preds==labels)/(preds==labels).size
    #print("Acc:", score*100) 
    #acc.append(score*100)
    #s=s+np.sum(preds==labels)
    #t=t+(preds==labels).size

In [ ]:
preds[0,:]

In [ ]:
from PIL import Image
images = imageio.imread('000026.jpg')
img = Image.fromarray(images, 'RGB')
img.show()